In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
from sklearn.feature_extraction.text import CountVectorizer
from xgboost import XGBClassifier
import pickle
import re
from sklearn.metrics import r2_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [58]:
data = pd.read_csv(r"/content/amazon_alexa.tsv", delimiter = '\t', quoting = 3)

In [59]:
data.dropna(inplace=True)

In [60]:
data['length'] = data['verified_reviews'].apply(len)

In [61]:
neg_reviews = " ".join([review for review in data[data['feedback'] == 0]['verified_reviews']])
neg_reviews = neg_reviews.lower().split()

pos_reviews = " ".join([review for review in data[data['feedback'] == 1]['verified_reviews']])
pos_reviews = pos_reviews.lower().split()

#Finding words from reviews which are present in that feedback category only
unique_negative = [x for x in neg_reviews if x not in pos_reviews]
unique_negative = " ".join(unique_negative)

unique_positive = [x for x in pos_reviews if x not in neg_reviews]
unique_positive = " ".join(unique_positive)

In [62]:
corpus = []
stemmer = PorterStemmer()
for i in range(0, data.shape[0]):
  review = re.sub('[^a-zA-Z]', ' ', data.iloc[i]['verified_reviews'])
  review = review.lower().split()
  review = [stemmer.stem(word) for word in review if not word in STOPWORDS]
  review = ' '.join(review)
  corpus.append(review)

In [63]:
#Storing independent and dependent variables in X and y
X = corpus
y = data['feedback']

In [70]:
from sklearn.pipeline import Pipeline,make_pipeline

In [71]:
steps = [
    ('vectorizer', CountVectorizer(max_features = 2500)),
    ('scaler', MaxAbsScaler()),
    ('xgboost', XGBClassifier())
]

In [72]:
pipeline = Pipeline(steps)

# Example usage (assuming you have your data as X and target variable as y)
pipeline.fit(X, y)
#predictions = pipeline.predict(X_test)

Pipeline(steps=[('vectorizer', CountVectorizer(max_features=2500)),
                ('scaler', MaxAbsScaler()),
                ('xgboost',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=None, ...))])

In [73]:
X_test=pd.read_csv('/content/SentimentBulk.csv')

In [74]:
pickle.dump(pipeline, open('/content/sample_data.pkl', 'wb'))

In [75]:
pipeline.predict(X_test.values[6])

array([1])